In [1]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

In [2]:
# llm = OpenAI(temperature=0)

openai_api_base_address = "http://192.168.3.20:20000/v1"

llm = OpenAI(openai_api_key = "aaabbbcccdddeeefffedddsfasdfasdf", 
        openai_api_base = openai_api_base_address,
        model_name = "vicuna-13b-v1.5")

In [3]:
import os

os.environ["SERPAPI_API_KEY"] = ""

tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [4]:
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

In [5]:
prompt = hub.pull("hwchase17/react")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [6]:
llm_with_stop = llm.bind(stop=["\nObservation"])

In [7]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

In [8]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...
First, I need to find out who Leo DiCaprio's girlfriend is. Then, I can find her current age. Finally, I will raise her age to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"Vittoria CerettiVittoria Ceretti is Leo DiCaprio's girlfriend, according to the internet.

Action: Search
Action Input: "Vittoria Ceretti age"25 years25 is Vittoria Ceretti's current age.

Action: Calculator
Action Input: 25 ^ 0.43Answer: 3.9912984526580783.991298452658078 is Vittoria Ceretti's age raised to the 0.43 power.

Final Answer: Vittoria Ceretti is Leo DiCaprio's girlfriend and her age raised to the 0.43 power is 3.991298452658078.

> Finished chain.


{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': "Vittoria Ceretti is Leo DiCaprio's girlfriend and her age raised to the 0.43 power is 3.991298452658078."}